In [ ]:
pip install tensorflow numpy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Flatten
from tensorflow.keras.models import Sequential

# Define the dimensions of the noise vector and image
NOISE_DIM = 100
IMAGE_SHAPE = (28, 28, 1)

def build_generator():
    model = Sequential()
    model.add(Dense(128, input_dim=NOISE_DIM))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(784, activation='sigmoid'))
    model.add(Reshape(IMAGE_SHAPE))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=IMAGE_SHAPE))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def train_gan(generator, discriminator, gan, x_train, epochs=10000, batch_size=64):
    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_images = x_train[idx]
        fake_images = generator.predict(np.random.randn(batch_size, NOISE_DIM))
        
        d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train Generator
        noise = np.random.randn(batch_size, NOISE_DIM)
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
        
        if epoch % 1000 == 0:
            print(f"Epoch {epoch} | Discriminator Loss: {d_loss[0]} | Generator Loss: {g_loss}")

            # Save generated images for visualization
            if epoch % 10000 == 0:
                plot_generated_images(generator, epoch)

def plot_generated_images(generator, epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.randn(examples, NOISE_DIM)
    generated_images = generator.predict(noise)
    
    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i].reshape(IMAGE_SHAPE), interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'gan_generated_image_{epoch}.png')
    plt.close()

# Load MNIST data
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = (x_train / 255.0).reshape(-1, 28, 28, 1).astype(np.float32)

# Build and compile models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Train GAN
train_gan(generator, discriminator, gan, x_train, epochs=10000, batch_size=64)
